# Load dataset.

## Go to the dataset directory.

In [1]:
%cd dataset/
!ls

[WinError 2] 系统找不到指定的文件。: 'dataset/'
C:\Users\wangy\NaturalLanguageProcessing\Epoch3-Sigmoid


'ls' is not recognized as an internal or external command,
operable program or batch file.


## Import the pandas library and perform data reading for analysis.

In [2]:
import pandas as pd
train = pd.read_csv('train.csv', sep='\t', header=None)
test = pd.read_csv('test.csv', sep='\t', header=None)

## Adding column names to data.

In [3]:
train.columns = ["text",'labels']
test.columns = ["text",'labels']

## The first 10 texts of the training set are read in the format "text, labels". The labels may contain multiple sentiment categories, each sentiment is separated by ','.

In [4]:
train.head(10)

,text,labels
0,My favourite food is anything I didn't have to...,27
1,"Now if he does off himself, everyone will thin...",27
2,WHY THE FUCK IS BAYLESS ISOING,2
3,To make her feel threatened,14
4,Dirty Southern Wankers,3
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26
6,Yes I heard abt the f bombs! That has to be wh...,15
7,We need more boards and to create a bit more s...,"8,20"
8,Damn youtube and outrage drama is super lucrat...,0
9,It might be linked to the trust factor of your...,27


In [5]:
test.head(10)

,text,labels
0,I’m really sorry about your situation :( Altho...,25
1,It's wonderful because it's awful. At not with.,0
2,"Kings fan here, good luck to you guys! Will be...",13
3,"I didn't know that, thank you for teaching me ...",15
4,They got bored from haunting earth for thousan...,27
5,Thank you for asking questions and recognizing...,15
6,You’re welcome,15
7,100%! Congrats on your job too!,15
8,I’m sorry to hear that friend :(. It’s for the...,24
9,"Girlfriend weak as well, that jump was pathetic.",25


# Install and import related libraries.

In [6]:
!pip install --upgrade paddlenlp

In [7]:
!pip install paddlepaddle

In [8]:
import os
import paddle
import paddlenlp

D:\anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tensorflow\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Data pre-processing

## For the 28 micro-sentiment multi-label classification scenario, where a sentence may correspond to multiple sentiment category labels, the sentiment labels of the dataset need to be transformed using One-Hot coding first, with "0" indicating absence and "1" indicating presence for each sentiment.

### Create sentiment label mapping relationships.

In [9]:
label_vocab = {
    0: "admiration",
    1: "amusement",
    2: "anger",
    3: "annoyance",
    4: "approval",
    5: "caring",
    6: "confusion",
    7: "curiosity",
    8: "desire",
    9: "disappointment",
    10: "disapproval",
    11: "disgust",
    12: "embarrassment",
    13: "excitement",
    14: "fear",
    15: "gratitude",
    16: "grief",
    17: "joy",
    18: "love",
    19: "nervousness",
    20: "optimism",
    21: "pride",
    22: "realization",
    23: "relief",
    24: "remorse",
    25: "sadness",
    26: "surprise",
    27: "neutral"
}

### Customize the dataset, read the data file, create the dataset and define the data type as MapDataset.

In [10]:
import re

from paddlenlp.datasets import load_dataset

# Clear invalid characters
def clean_text(text):
    text = text.replace("\r", "").replace("\n", "")
    text = re.sub(r"\\n\n", ".", text)
    return text

# Define the read data set function
def read_custom_data(filepath, is_one_hot=True):
    f = open(filepath)
    while True:
        line = f.readline()
        if not line:
            break
        data = line.strip().split('\t')
        # One-hot processing for 28 types of micro sentiment tags
        if is_one_hot:
            labels = [float(1) if str(i) in data[1].split(',') else float(0) for i in range(28)]  # 28 types
        else:
            labels = [int(d) for d in data[1].split(',')]
        yield {"text": clean_text(data[0]), "labels": labels}
    f.close()

In [11]:
# load_dataset() to Create dataset.
# lazy=False，The dataset is returned as a MapDataset type.
# Pre-processing of training and validation sets.
train_ds = load_dataset(read_custom_data, filepath='train.csv', lazy=False) 
test_ds = load_dataset(read_custom_data, filepath='test.csv', lazy=False)

### Print dataset.

In [12]:
print("datatype:", type(train_ds))
print("training dataset example:", train_ds[0])
print("testing dataset example:", test_ds[0])

datatype: <class 'paddlenlp.datasets.dataset.MapDataset'>
training dataset example: {'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}
testing dataset example: {'text': 'I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!', 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]}


## Load Chinese ERNIE 3.0 pre-training model and word splitter

In [13]:
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ernie-3.0-medium-zh"   # ERNIE3.0 model
num_classes = 28  # 28 classification mission
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2023-04-13 17:54:47,838] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-3.0-medium-zh'.
[2023-04-13 17:54:47,839] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27"


## Process the raw data into a model-acceptable format.

In [14]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# Data pre-processing function to convert text into integer sequences using a word splitter.
def preprocess_function(examples, tokenizer, max_seq_length):
    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)
    result["labels"] = examples["labels"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=64)
train_ds = train_ds.map(trans_func)
test_ds = test_ds.map(trans_func)

# function is constructed to extend the different length sequences to the maximum length of the data in the batch, and then stack the data.
collate_fn = DataCollatorWithPadding(tokenizer)

# Define the BatchSampler, select the batch size and whether to randomly jumble the DataLoader.
train_batch_sampler = BatchSampler(train_ds, batch_size=32, shuffle=True)
test_batch_sampler = BatchSampler(test_ds, batch_size=16, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
test_data_loader = DataLoader(dataset=test_ds, batch_sampler=test_batch_sampler, collate_fn=collate_fn)

## Define model validation metrics.

In [15]:
import numpy as np
import sklearn
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from paddle.metric import Metric

# Customize MultiLabelReport evaluation metrics.
class MultiLabelReport(Metric):
    """
    AUC and F1 Score for multi-label text classification task.
    """

    def __init__(self, name='MultiLabelReport', average='micro'):
        super(MultiLabelReport, self).__init__()
        self.average = average
        self._name = name
        self.reset()

    def f1_score(self, y_prob):
        '''
        Returns the f1 score by searching the best threshhold
        '''
        best_score = 0
        for threshold in [i * 0.01 for i in range(100)]:
            self.y_pred = y_prob > threshold
            score = sklearn.metrics.f1_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
            if score > best_score:
                best_score = score
                precison = precision_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
                recall = recall_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
        return best_score, precison, recall

    def reset(self):
        """
        Resets all of the metric state.
        """
        self.y_prob = None
        self.y_true = None

    def update(self, probs, labels):
        if self.y_prob is not None:
            self.y_prob = np.append(self.y_prob, probs.numpy(), axis=0)
        else:
            self.y_prob = probs.numpy()
        if self.y_true is not None:
            self.y_true = np.append(self.y_true, labels.numpy(), axis=0)
        else:
            self.y_true = labels.numpy()

    def accumulate(self):
        auc = roc_auc_score(
            y_score=self.y_prob, y_true=self.y_true, average=self.average)
        f1_score, precison, recall = self.f1_score(y_prob=self.y_prob)
        return auc, f1_score, precison, recall

    def name(self):
        """
        Returns metric name
        """
        return self._name

# Building the training model.

## Select an optimization strategy and run configuration.

In [45]:
import time
import paddle.nn.functional as F

# AdamW optimizer, cross-entropy loss function, custom MultiLabelReport evaluation metrics.
optimizer = paddle.optimizer.Adamax(learning_rate=4e-5, parameters=model.parameters(), weight_decay=0.01)
criterion = paddle.nn.BCEWithLogitsLoss()
metric = MultiLabelReport()

## Model training and validation.

In [46]:
import paddle
import numpy as np
import paddle.nn.functional as F

# Build the validation set evaluate function.
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader, label_vocab, if_return_results=True):
    model.eval()
    metric.reset()
    losses = []
    results = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.sigmoid(logits)
        losses.append(loss.numpy())
        metric.update(probs, labels)
        if if_return_results:
            probs = probs.tolist()
            for prob in probs:
                result = []
                for c, pred in enumerate(prob):
                    if pred > 0.5:
                        result.append(label_vocab[c])
                results.append(','.join(result))

    auc, f1_score, precison, recall = metric.accumulate()
    print("eval loss: %.5f, auc: %.5f, f1 score: %.5f, precison: %.5f, recall: %.5f" %
          (np.mean(losses), auc, f1_score, precison, recall))
    model.train()
    metric.reset()
    if if_return_results:
        return results
    else:
        return f1_score

In [47]:
epochs = 3 # training times
ckpt_dir = "ernie_ckpt" # Folder for saving model parameters during training

global_step = 0  # Number of iterations
tic_train = time.time()
best_f1_score = 0

# Model Training
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # Calculate model output, loss function value, classification probability value, accuracy, f1 score.
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.sigmoid(logits)
        metric.update(probs, labels)
        auc, f1_score, _, _ = metric.accumulate()

        # Print the loss function value, accuracy, f1 score, and computation speed for each 10 iterations.
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, auc: %.5f, f1 score: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, auc, f1_score,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # Reverse gradient passback with updated parameters.
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        
        # Every 40 iterations, evaluate the current trained model, save the current best model parameters and word list of the word splitter, etc.
        if global_step % 40 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            eval_f1_score = evaluate(model, criterion, metric, test_data_loader, label_vocab, if_return_results=False)
            if eval_f1_score > best_f1_score:
                best_f1_score = eval_f1_score
                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

global step 10, epoch: 1, batch: 10, loss: 0.14621, auc: 0.78937, f1 score: 0.32947, speed: 0.23 step/s
global step 20, epoch: 1, batch: 20, loss: 0.14557, auc: 0.79827, f1 score: 0.34981, speed: 0.21 step/s
global step 30, epoch: 1, batch: 30, loss: 0.14175, auc: 0.79670, f1 score: 0.34879, speed: 0.21 step/s
global step 40, epoch: 1, batch: 40, loss: 0.13866, auc: 0.80003, f1 score: 0.36761, speed: 0.20 step/s


[2023-04-13 23:49:11,786] [    INFO] - Configuration saved in ernie_ckpt\config.json


eval loss: 0.13796, auc: 0.80761, f1 score: 0.38273, precison: 0.43718, recall: 0.34034


[2023-04-13 23:49:13,523] [    INFO] - tokenizer config file saved in ernie_ckpt\tokenizer_config.json
[2023-04-13 23:49:13,524] [    INFO] - Special tokens file saved in ernie_ckpt\special_tokens_map.json


global step 50, epoch: 1, batch: 50, loss: 0.13913, auc: 0.81349, f1 score: 0.40121, speed: 0.04 step/s
global step 60, epoch: 1, batch: 60, loss: 0.13333, auc: 0.81883, f1 score: 0.40302, speed: 0.19 step/s
global step 70, epoch: 1, batch: 70, loss: 0.15155, auc: 0.81531, f1 score: 0.39302, speed: 0.19 step/s
global step 80, epoch: 1, batch: 80, loss: 0.16581, auc: 0.80832, f1 score: 0.38055, speed: 0.19 step/s


...

global step 1500, epoch: 1, batch: 1500, loss: 0.14202, auc: 0.79405, f1 score: 0.35016, speed: 0.08 step/s
global step 1510, epoch: 1, batch: 1510, loss: 0.14160, auc: 0.79271, f1 score: 0.36167, speed: 0.07 step/s
global step 1520, epoch: 1, batch: 1520, loss: 0.14990, auc: 0.79373, f1 score: 0.35615, speed: 0.07 step/s
eval loss: 0.14006, auc: 0.79546, f1 score: 0.37140, precison: 0.45130, recall: 0.31553
global step 1530, epoch: 2, batch: 3, loss: 0.15453, auc: 0.77961, f1 score: 0.35473, speed: 0.03 step/s
global step 1540, epoch: 2, batch: 13, loss: 0.13862, auc: 0.78297, f1 score: 0.36129, speed: 0.06 step/s
global step 1550, epoch: 2, batch: 23, loss: 0.15780, auc: 0.78209, f1 score: 0.34732, speed: 0.07 step/s
global step 1560, epoch: 2, batch: 33, loss: 0.14480, auc: 0.78127, f1 score: 0.35145, speed: 0.06 step/s
eval loss: 0.14036, auc: 0.79656, f1 score: 0.37020, precison: 0.43478, recall: 0.32233
global step 1570, epoch: 2, batch: 43, loss: 0.11984, auc: 0.79332, f1 score:

global step 2140, epoch: 2, batch: 613, loss: 0.14233, auc: 0.78193, f1 score: 0.32005, speed: 0.03 step/s
global step 2150, epoch: 2, batch: 623, loss: 0.15025, auc: 0.79028, f1 score: 0.32531, speed: 0.03 step/s
global step 2160, epoch: 2, batch: 633, loss: 0.13308, auc: 0.78741, f1 score: 0.32321, speed: 0.03 step/s
eval loss: 0.14120, auc: 0.79605, f1 score: 0.36182, precison: 0.40954, recall: 0.32406
global step 2170, epoch: 2, batch: 643, loss: 0.14489, auc: 0.78970, f1 score: 0.35223, speed: 0.02 step/s
global step 2180, epoch: 2, batch: 653, loss: 0.14817, auc: 0.80053, f1 score: 0.35886, speed: 0.03 step/s
global step 2190, epoch: 2, batch: 663, loss: 0.13438, auc: 0.80094, f1 score: 0.35107, speed: 0.03 step/s
global step 2200, epoch: 2, batch: 673, loss: 0.16592, auc: 0.80190, f1 score: 0.35053, speed: 0.03 step/s
eval loss: 0.14096, auc: 0.79625, f1 score: 0.36364, precison: 0.41448, recall: 0.32391
global step 2210, epoch: 2, batch: 683, loss: 0.14165, auc: 0.78487, f1 sco

global step 2780, epoch: 2, batch: 1253, loss: 0.15496, auc: 0.77081, f1 score: 0.33539, speed: 0.02 step/s
global step 2790, epoch: 2, batch: 1263, loss: 0.16864, auc: 0.76859, f1 score: 0.32132, speed: 0.02 step/s
global step 2800, epoch: 2, batch: 1273, loss: 0.15599, auc: 0.77384, f1 score: 0.32799, speed: 0.02 step/s
eval loss: 0.14075, auc: 0.79954, f1 score: 0.36370, precison: 0.41310, recall: 0.32485
global step 2810, epoch: 2, batch: 1283, loss: 0.15699, auc: 0.76695, f1 score: 0.31206, speed: 0.01 step/s
global step 2820, epoch: 2, batch: 1293, loss: 0.14233, auc: 0.78408, f1 score: 0.31724, speed: 0.02 step/s
global step 2830, epoch: 2, batch: 1303, loss: 0.15631, auc: 0.77773, f1 score: 0.31365, speed: 0.02 step/s
global step 2840, epoch: 2, batch: 1313, loss: 0.13347, auc: 0.78787, f1 score: 0.32941, speed: 0.02 step/s
eval loss: 0.14101, auc: 0.79972, f1 score: 0.35999, precison: 0.40171, recall: 0.32612
global step 2850, epoch: 2, batch: 1323, loss: 0.13359, auc: 0.77601

global step 3420, epoch: 3, batch: 366, loss: 0.13679, auc: 0.77148, f1 score: 0.32551, speed: 0.02 step/s
global step 3430, epoch: 3, batch: 376, loss: 0.12732, auc: 0.78306, f1 score: 0.35107, speed: 0.01 step/s
global step 3440, epoch: 3, batch: 386, loss: 0.14287, auc: 0.78730, f1 score: 0.35026, speed: 0.02 step/s
eval loss: 0.13997, auc: 0.79913, f1 score: 0.36899, precison: 0.44363, recall: 0.31585
global step 3450, epoch: 3, batch: 396, loss: 0.13736, auc: 0.79229, f1 score: 0.34643, speed: 0.01 step/s
global step 3460, epoch: 3, batch: 406, loss: 0.13365, auc: 0.78970, f1 score: 0.32743, speed: 0.02 step/s
global step 3470, epoch: 3, batch: 416, loss: 0.12591, auc: 0.79070, f1 score: 0.33350, speed: 0.02 step/s
global step 3480, epoch: 3, batch: 426, loss: 0.14059, auc: 0.78973, f1 score: 0.32669, speed: 0.02 step/s
eval loss: 0.13992, auc: 0.79841, f1 score: 0.37017, precison: 0.44425, recall: 0.31727
global step 3490, epoch: 3, batch: 436, loss: 0.14202, auc: 0.78054, f1 sco

global step 4060, epoch: 3, batch: 1006, loss: 0.14225, auc: 0.79685, f1 score: 0.33333, speed: 0.01 step/s
global step 4070, epoch: 3, batch: 1016, loss: 0.15236, auc: 0.79192, f1 score: 0.33147, speed: 0.01 step/s
global step 4080, epoch: 3, batch: 1026, loss: 0.13528, auc: 0.79888, f1 score: 0.35252, speed: 0.02 step/s
eval loss: 0.14005, auc: 0.80204, f1 score: 0.36561, precison: 0.42845, recall: 0.31885
global step 4090, epoch: 3, batch: 1036, loss: 0.13537, auc: 0.79791, f1 score: 0.34835, speed: 0.01 step/s
global step 4100, epoch: 3, batch: 1046, loss: 0.13173, auc: 0.78568, f1 score: 0.34184, speed: 0.02 step/s
global step 4110, epoch: 3, batch: 1056, loss: 0.13301, auc: 0.79410, f1 score: 0.35608, speed: 0.02 step/s
global step 4120, epoch: 3, batch: 1066, loss: 0.15780, auc: 0.79774, f1 score: 0.35846, speed: 0.02 step/s
eval loss: 0.13967, auc: 0.80196, f1 score: 0.36843, precison: 0.43597, recall: 0.31901
global step 4130, epoch: 3, batch: 1076, loss: 0.13948, auc: 0.79532

## Model Validation Performance Results

In [48]:
# Load the optimal parameters of the trained model.
model.set_dict(paddle.load('ernie_ckpt/model_state.pdparams'))

# Load the parameters of the previously trained model.
# model.set_dict(paddle.load('/home/aistudio/work/model_state.pdparams'))

# Model Validation.
print("ERNIE 3.0 performance on GoEmotions micro-emotion 28 classification test set：", end= " ")
results = evaluate(model, criterion, metric, test_data_loader, label_vocab)

ERNIE 3.0 performance on GoEmotions micro-emotion 28 classification test set： eval loss: 0.13739, auc: 0.80681, f1 score: 0.39162, precison: 0.44919, recall: 0.34713


## 28 Multi-Label Groups of "Micro" Emotions Predicting Demo

In [49]:
# Define data loading and processing functions
from paddlenlp.data import JiebaTokenizer, Pad, Stack, Tuple, Vocab
def convert_example(example, tokenizer, max_seq_length=64, is_test=False):
    qtconcat = example["text"]
    encoded_inputs = tokenizer(text=qtconcat, max_seq_len=max_seq_length)
    input_ids = encoded_inputs["input_ids"]
    token_type_ids = encoded_inputs["token_type_ids"]
    if not is_test:
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    else:
        return input_ids, token_type_ids

# Define the model prediction function
def predict(model, data, tokenizer, label_vocab, batch_size=1, max_seq=64):
    examples = []
    # Process input data (the list form) into a format acceptable to the model
    for text in data:
        input_ids, segment_ids = convert_example(
            text,
            tokenizer,
            max_seq_length=max_seq,
            is_test=True)
        examples.append((input_ids, segment_ids))

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input id
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # segment id
    ): fn(samples)

    # Seperates data into some batches.
    batches = []
    one_batch = []
    for example in examples:
        one_batch.append(example)
        if len(one_batch) == batch_size:
            batches.append(one_batch)
            one_batch = []
    if one_batch:
        # The last batch whose size is less than the config batch_size setting.
        batches.append(one_batch)

    results = []
    model.eval()
    for batch in batches:
        input_ids, segment_ids = batchify_fn(batch)
        input_ids = paddle.to_tensor(input_ids)
        segment_ids = paddle.to_tensor(segment_ids)
        logits = model(input_ids, segment_ids)
        probs = F.relu(logits)
        probs = probs.tolist()
        # The results were processed by selecting the sentiment categories with probability greater than 0.5
        for prob in probs:
            result = []
            for c, pred in enumerate(prob):
                if pred > 0.5:
                    result.append(label_vocab[c])
            results.append(','.join(result))
    return results  # Return the predeicted results

## Prediction Results

In [51]:
# Define the text data to be subjected to micro-sentiment analysis
data = [
    # 0 admiration
    {"text": 'You do a great job!'},
    # 1 amusement
    {"text": 'Lets have fun!'},
    # 2 anger
    {"text":"You shut your mouth"},
    # 3 annoyance
    {"text": 'You are so annoyed'},
    # 4 approval
    {"text": 'You are allowed to do this'},
    # 5 caring & # 7 Curiosity
    {"text": 'Are you feeling well?'},
    # 6 confusion
    {"text": 'This problem is so hard and I cannot solve this problem'},
    # 7 curiosity
    {"text":'Why would I do that?'},
    # 8 desire
    {"text": 'I want this gift so much'},
    # 9 disappointment
    {"text": 'I am very disappointed by everything you have done to me'},
    # 10 disapproval
    {"text": 'You are not admitted to the college.'},
    # 11 disgust
    {"text": 'Thats absolutely disgusting.'},
    # 12 embarrassment
    {"text": 'Thats so embarrassing.'},
    # 13 excitement
    {"text": 'I cannot wait to join the party'},
    # 14 fear
    {"text": 'I am so scared of skydiving'},
    # 15 gratitude
    {"text":"Thank you."},
    # 16 grief
    {"text": 'I feel sorry that your grandpa passed away'},
    # 17 joy
    {"text": 'Happy Birthday'},
    # 18 love
    {"text": 'I love you so much'},
    # 19 nervousness
    {"text": 'I am so nervous.'},
    # 20 neutral
    {"text": 'It is just so so.'},
    # 21 optimism
    {"text": 'Successful people only focus on giving their best effort.'},
    # 22 pride
    {"text": 'I am so proud of you.'},
    # 23 realization
    {"text": 'I just found you like basketball too.'},
    # 24 relief 
    {"text": 'Do not worry and take it easy.'},
    # 25 remorse
    {"text": 'I am guilty.'},
    # 26 sadness
    {"text": 'I am so sad.'},
    # 27 surprise
    {"text": 'I am so surprised that you made it.'}
]

# Model Prediction
labels =  predict(model, data, tokenizer, label_vocab, batch_size=1)

# Output of predicted results
for idx, text in enumerate(data):
    print('Text: {} \t Labels: {}'.format(text['text'], labels[idx]))

Text: You do a great job! 	 Labels: 
Text: Lets have fun! 	 Labels: 
Text: You shut your mouth 	 Labels: 
Text: You are so annoyed 	 Labels: 
Text: You are allowed to do this 	 Labels: 
Text: Are you feeling well? 	 Labels: 
Text: This problem is so hard and I cannot solve this problem 	 Labels: 
Text: Why would I do that? 	 Labels: 
Text: I want this gift so much 	 Labels: 
Text: I am very disappointed by everything you have done to me 	 Labels: 
Text: You are not admitted to the college. 	 Labels: 
Text: Thats absolutely disgusting. 	 Labels: 
Text: Thats so embarrassing. 	 Labels: 
Text: I cannot wait to join the party 	 Labels: 
Text: I am so scared of skydiving 	 Labels: 
Text: Thank you. 	 Labels: 
Text: I feel sorry that your grandpa passed away 	 Labels: 
Text: Happy Birthday 	 Labels: 
Text: I love you so much 	 Labels: 
Text: I am so nervous. 	 Labels: 
Text: It is just so so. 	 Labels: 
Text: Successful people only focus on giving their best effort. 	 Labels: 
Text: I am so 